In [41]:
import sys
# win:
# project_path = 'D:/Code/repos/gad1b-redistribution/src'
# wsl:
project_path = '/mnt/d/Code/repos/gad1b-redistribution'
sys.path.insert(1, f'{project_path}/src')

from pw_registratoin.core import *
from pw_registratoin.viewers import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Test Simple Volume Manipulations
Here you will split an image into voxels and will create a tiled output view.

In [15]:
test_image = f'{project_path}/data/test/sequence01.tif'
img = Image(test_image, [1,1,1])
img.shape

(200, 400, 250)

In [20]:
voxels = img.split(50,25)

In [21]:
filename = f'{project_path}/data/test/test_output_overlap.tif'
VoxelView(voxels,20).write_volume(filename)

## Test Registration ( needs ants )

Here you will register volumes and see how it's going. 


First pre-register the two files. Get the images : 

In [25]:
fixed = Image([0.658, 0.23, 0.23], filename = f'{project_path}/data/raw/img/cropped/1-1Z8J_cropped_from51_left.tif')
moving = Image([0.658, 0.23, 0.23], filename = f'{project_path}/data/raw/img/cropped/1-1Z8M_cropped_from51_left.tif')

split them into 1 giant voxel .. because I only implemented registration for voxels :D 

In [27]:
voxels_f = fixed.split([123,1024,512],0)
voxels_m = moving.split([138,1024,512],0)

arrange voxels in pairs and register the pairs 

In [35]:
pair = VoxelPair(voxels_f[0],voxels_m[0])
pair.register()

save the resulting registered image to look at it in ImageJ 

In [37]:
tif.imsave(f'{project_path}/data/test/test_registration.tif', pair.wrapped, imagej=True)

Cool! Now use the pre-registered image and fine-tune the registration using smaller voxels : 

In [43]:
prereg = Image([0.658, 0.23, 0.23],img = pair.wrapped)

In [51]:
voxels_f = fixed.split([20,102,102],0)
voxels_m = prereg.split([20,102,102],0)

In [52]:
pairs = [VoxelPair(voxf,voxm) for voxf,voxm in zip(voxels_f,voxels_m)]

In [55]:
for pair in pairs: 
    pair.register()

Output the registered and the split image and enjoy ! 

In [61]:
VoxelPairView(pairs, 10).write_volume(f'{project_path}/data/test/test_pw_registration_moving.tif')
VoxelView(voxels_f,10).write_volume(f'{project_path}/data/test/test_pw_registration_fixed.tif')